# Getting Started With TorchServe

## Install PyTorch and TorchServe and conduct an inference.

This introductory lab will prepare your environment and test that TorchServe is working.

### Prepare the TorchServe environment on SageMaker
1. Use Amazon Corretto to install Java 11:

In [ ]:
%%bash
sudo rpm --import https://yum.corretto.aws/corretto.key 
sudo curl -L -o /etc/yum.repos.d/corretto.repo https://yum.corretto.aws/corretto.repo
sudo yum install -y java-11-amazon-corretto-devel

2. Verify the Java version and that JAVA_HOME is properly set.

In [ ]:
!java -version
!echo $JAVA_HOME

3. Use pip to install TorchServe and the model archiver:

In [ ]:
!pip install torch torchtext torchvision sentencepiece psutil future
!pip install torchserve torch-model-archiver

4. If the serve subdirectory already exists, remove it. And, then clone the TorchServe repository into a new serve subdirectory. 

In [ ]:
%%bash
if [ -d "serve" ]; then
    rm -r -f serve
fi
git clone https://github.com/pytorch/serve.git serve

### Store a model

5. Create thde model_store directory which is reference via a parameter by the model_archiver. If the model_store subdirectory already exists, remove all the files. Else, create a subdirectory in which to store your models. 

In [ ]:
%%bash
if [ -d "model_store" ]; then
    rm -f model_store/*    
else
    mkdir model_store
fi

6. Download a trained model. The DenseNet model is one of the PyTorch TorchVision [models](https://pytorch.org/docs/stable/torchvision/models.html). You can read more about it at [arxiv.org](https://arxiv.org/abs/1608.06993).

In [ ]:
%%bash
wget -q https://download.pytorch.org/models/densenet161-8d451a50.pth
ls *.pth

7. Archive the model using the TorchServe model archiver using parameters for the model name, versin, the model file provided by the examples, the serialized densenet model. You can find more information on using the model archiver at [Torch Model Archiver for TorchServe](https://github.com/pytorch/serve/blob/master/model-archiver/README.md). We conclude this step by listing the archived model file. 

In [ ]:
!torch-model-archiver \
   --model-name densenet161 \
   --version 1.0 \
   --model-file serve/examples/image_classifier/densenet_161/model.py \
   --serialized-file densenet161-8d451a50.pth \
   --extra-files serve/examples/image_classifier/index_to_name.json \
   --handler image_classifier

!ls *.mar

8. Move the archived model into the model_store subdirectory.

In [ ]:
!mv *.mar model_store

### Start TorchServe
9. Once you archive and store the model, you will start the TorchServe server in the background. Once it has started, TorchServe listens for your inference requests.

In [ ]:
%%bash
torchserve --start --model-store model_store --models densenet161=densenet161.mar </dev/null &>/dev/null &

10. Check status of TorchServe process. The status should read "Healthy".

In [ ]:
!curl http://localhost:8080/ping

11. Show the model(s) being served.

In [ ]:
!curl http://localhost:8081/models

### Perform inference
To test the TorchServe model server, you just need to send a request to the prediction API. 
12. Use curl to download an image. In this case, it is the image of a kitten. We use the -O flag to name the download 'kitten.jpg'

In [ ]:
!curl -O https://s3.amazonaws.com/model-server/inputs/kitten.jpg

13. Now that you have an image, you use curl to send POST to the TorchServe predict endpoint with the kitten's image. The predictions endpoint returns a prediction response in JSON. With the kitten image, you will see several different types along with the confidence of each prediction.

In [ ]:
!curl -X POST http://localhost:8080/predictions/densenet161 -T kitten.jpg

14. When you have completed running inferences, you may stop the server by executing the torchserve command with the --stop flag.

In [ ]:
!torchserve --stop